<a href="https://colab.research.google.com/github/drvoss/Colab-Notebooks/blob/master/CNN_regressive_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
!tar -zxvf lfw-deepfunneled.tgz
!mkdir lfw-deepfunneled/train
!mv lfw-deepfunneled/[A-W]* lfw-deepfunneled/train
!mkdir lfw-deepfunneled/test
!mv lfw-deepfunneled/[X-Z]* lfw-deepfunneled/test

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)
import tqdm

from torchvision.datasets import ImageFolder
from torchvision import transforms

class DownSizedPairImageFolder(ImageFolder):
  def __init__(self, root, transform=None,
               large_size=128, small_size=32, **kwds):
    super().__init__(root, transform=transform, **kwds)
    self.large_resizer = transforms.Resize(large_size)
    self.small_resizer = transforms.Resize(small_size)

def __getitem__(self, index):
  path, _ = self.imgs[index]
  img = self.loader(path)
  # 읽은 이미지를 128×128픽셀과 32×32픽셀로 리사이즈
  large_img = self.large_resizer(img)
  small_img = self.small_resizer(img)
  # 기타 변환 적용
  if self.transform is not None:
    large_img = self.transform(large_img)
    small_img = self.transform(small_img)
  # 32픽셀의 이미지와 128픽셀의 이미지 반환
  return small_img, large_img

train_data = DownSizedPairImageFolder("/content/lfw-deepfunneled/train",
transform=transforms.ToTensor())
test_data = DownSizedPairImageFolder("/content/lfw-deepfunneled/test",
transform=transforms.ToTensor())
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)

net = nn.Sequential(
  nn.Conv2d(3, 256, 4, stride=2, padding=1),
  nn.ReLU(),
  nn.BatchNorm2d(256),
  nn.Conv2d(256, 512, 4, stride=2, padding=1),
  nn.ReLU(),
  nn.BatchNorm2d(512),
  nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
  nn.ReLU(),
  nn.BatchNorm2d(256),
  nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
  nn.ReLU(),
  nn.BatchNorm2d(128),
  nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
  nn.ReLU(),
  nn.BatchNorm2d(64),
  nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1)
)

import math
def psnr(mse, max_v=1.0):
  return 10 * math.log10(max_v**2 / mse)

# 평가 헬퍼 함수
def eval_net(net, data_loader, device="cpu"):
  # Dropout 및 BatchNorm을 무효화
  net.eval()
  ys = []
  ypreds = []
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
      y_pred = net(x)
    ys.append(y)
    ypreds.append(y_pred)
  # 미니 배치 단위로 예측 결과 등을 하나로 모은다
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  # 예측 정확도(MSE) 계산
  score = nn.functional.mse_loss(ypreds, ys).item()
  return score

# 훈련 헬퍼 함수
def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam
              , loss_fn=nn.MSELoss()
              , n_iter=10, device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  optimizer = optimizer_cls(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    # 신경망을 훈련 모드로 설정
    net.train()
    n = 0
    score = 0
    # 시간이 많이 걸리므로 tqdm를 이용해서
    # 진행 바 표시
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader),
      total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      y_pred = net(xx)
      loss = loss_fn(y_pred, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
    train_losses.append(running_loss / len(train_loader))
    # 검증 데이터의 훈련 정확도
    val_acc.append(eval_net(net, test_loader, device))
    # epoch의 결과 표시
    print(epoch, train_losses[-1],
    psnr(train_losses[-1]), psnr(val_acc[-1]), flush=True)
    
net.to("cuda:0")
train_net(net, train_loader, test_loader, device="cuda:0")

  0%|          | 0/409 [00:00<?, ?it/s]


RuntimeError: ignored